In [2]:
import jsonc

In [3]:
original = "loottables_original.json"

In [49]:
with open(original, "r") as file:
    loottable_dict = jsonc.load(file)

In [5]:
for key in loottable_dict:
    print(key)

MagicEffectsCount
RestrictedItems
ItemSets
LootTables


## MagicEffectsCount Functions
### Functions that change the ammount, and it's associated probability, of enchantments

In [6]:
def increase_enchant_count(n):
    """
    Increases all magic effects count by n.
    Changes probabilities only indirectly.
    """
    print("########################## BEFORE ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
        for i in range(len(probabilities)):
            probabilities[i][0] += 1
    
    print("########################## AFTER ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
    
    print("############################################################")
    print("##################### INCREASE COUNT #######################")
    print("############################################################")
    print()
    print()
    return 1

def modify_probabilities(p):
    """
    Takes p away from the first entry and adds 2*p/3 and p/3 to the second and last entry, respectively
    """
    print("########################## BEFORE ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
        probabilities[0][1] -= p
        probabilities[1][1] += (p//3) * 2
        probabilities[2][1] = 100 - probabilities[0][1] - probabilities[1][1]
    
    print("########################## AFTER ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
    
    print("############################################################")
    print("################### MODIFY PROBABILITIES ###################")
    print("############################################################")
    print()
    print()
    return 1

## ItemSets Functions
### Functions that create/delete item sets

###### Note: EpicLoot provides a setting to determine the item-to-material drop ratio. The only provided function to change item sets in this code aims to improve configuration in case you are running CLLC and want only materials dropping. In such cases, utilising solely the EpicLoot configuration, I was met with bosses trophies drop not scaling with starts. The solution was to manually change the loottables.json file.

In [28]:
def only_enchanting_mats():
    """
    Deletes all item sets that don't include enchanting mats
    """
    item_set_index = 0
    while item_set_index < len(loottable_dict["ItemSets"]):
        item_set = loottable_dict["ItemSets"][item_set_index]
        #print(item_set["Name"], [char_seq in item_set["Name"] for char_seq in ["Mats", "Runestone"]])
        if not any([char_seq in item_set["Name"] for char_seq in ["Mats", "Runestone"]]):
            print(f"{item_set['Name']} {' ' * (18-len(str(item_set['Name'])))} DELETED")
            loottable_dict["ItemSets"].pop(item_set_index)
        else:
            item_set_index += 1

In [29]:
only_enchanting_mats()

ModUtility          DELETED
Tier0Weapons        DELETED
Tier0Tools          DELETED
Tier0Armor          DELETED
Tier0Shields        DELETED
Tier0Everything     DELETED
Tier1Weapons        DELETED
Tier1Armor          DELETED
Tier1Tools          DELETED
Tier1Shields        DELETED
Tier1Everything     DELETED
TrollArmor          DELETED
Tier2Weapons        DELETED
Tier2Armor          DELETED
Tier2Shields        DELETED
Tier2Tools          DELETED
Tier2Everything     DELETED
Tier3Weapons        DELETED
Tier3Armor          DELETED
Tier3Shields        DELETED
Tier3Tools          DELETED
Tier3Everything     DELETED
Tier4Weapons        DELETED
Tier4Armor          DELETED
Tier4Shields        DELETED
Tier4Everything     DELETED
Tier5Weapons        DELETED
Tier5Armor          DELETED
Tier5Shields        DELETED
Tier5Everything     DELETED
Tier6Weapons        DELETED
Tier6Armor          DELETED
Tier6Shields        DELETED
Tier6Tools          DELETED
Tier6Everything     DELETED


In [63]:
for mat_tier in (loottable_dict['ItemSets'][0]["Loot"]):
    print(mat_tier["Item"])

Tier0Mats
Tier1Mats
Tier2Mats
Tier3Mats


In [30]:
for item_set in loottable_dict["ItemSets"]:
    print(item_set)

{'Name': 'EnchantingMats', 'Loot': [{'Item': 'Tier0Mats'}, {'Item': 'Tier1Mats'}, {'Item': 'Tier2Mats'}, {'Item': 'Tier3Mats'}]}
{'Name': 'Tier0Mats', 'Loot': [{'Item': 'DustMagic'}, {'Item': 'EssenceMagic'}, {'Item': 'ReagentMagic'}]}
{'Name': 'Tier1Mats', 'Loot': [{'Item': 'DustRare'}, {'Item': 'EssenceRare'}, {'Item': 'ReagentRare'}]}
{'Name': 'Tier2Mats', 'Loot': [{'Item': 'DustEpic'}, {'Item': 'EssenceEpic'}, {'Item': 'ReagentEpic'}]}
{'Name': 'Tier3Mats', 'Loot': [{'Item': 'DustLegendary'}, {'Item': 'EssenceLegendary'}, {'Item': 'ReagentLegendary'}]}
{'Name': 'EnchantingRunestones', 'Loot': [{'Item': 'RunestoneMagic'}, {'Item': 'RunestoneRare'}, {'Item': 'RunestoneEpic'}, {'Item': 'RunestoneLegendary'}]}
{'Name': 'Tier0Runestone', 'Loot': [{'Item': 'RunestoneMagic'}]}
{'Name': 'Tier1Runestone', 'Loot': [{'Item': 'RunestoneRare'}]}
{'Name': 'Tier2Runestone', 'Loot': [{'Item': 'RunestoneEpic'}]}
{'Name': 'Tier3Runestone', 'Loot': [{'Item': 'RunestoneLegendary'}]}


## Non-vanilla mobs

In [31]:
def add_mob_objects():
    """
    Creates mob objects for all mob ids added in the mobs_additions.txt file, each with their
    respective tier, also defined in the file
    """
    with open("mobs_additions.txt", "r") as mobs_file:
        for line in mobs_file:
            PRESENT = False
            if line[0] == "#" or line == "\n":
                continue
            mob_id, tier = line.replace("\n", "").split()
            for obj in  loottable_dict["LootTables"]:
                if mob_id == obj["Object"]:
                    PRESENT = True
                    break
            if PRESENT:
                print(f"{mob_id}{(20-len(mob_id)) * ' '}\t{tier}" + f"{10 * ' '}" + "ALREADY ADDED")
                continue
            print(f"{mob_id}{(20-len(mob_id)) * ' '}\t{tier}", end=" ")
            mob_obj = {}
            mob_obj["Object"] = mob_id
            mob_obj["RefObject"] = f"Tier{tier}Mob"
            loottable_dict["LootTables"].insert(8, mob_obj)
            print(mob_obj, f"{(65-len(str(mob_obj))) * ' '}ADDED")

In [32]:
add_mob_objects()

Fox_TW              	0          ALREADY ADDED
Razorback_TW        	0          ALREADY ADDED
BlackBear_TW        	1          ALREADY ADDED
GDAncientShaman_TW  	1          ALREADY ADDED
Crawler_TW          	2          ALREADY ADDED
HelWraith_TW        	2          ALREADY ADDED
RottingElk_TW       	3          ALREADY ADDED
GrizzlyBear_TW      	3          ALREADY ADDED
ObsidianGolem_TW    	4          ALREADY ADDED
Prowler_TW          	4          ALREADY ADDED


In [36]:
k = 0
for mob_object in loottable_dict['LootTables']:
    if k > 3:
        k = 0
        print()
    print(mob_object['Object'], end=" | ")
    k += 1

Tier0Mob | Tier1Mob | Tier2Mob | Tier3Mob | 
Tier4Mob | Tier5Mob | Tier6Mob | Tier7Mob | 
Greyling | Deer | Boar | Neck | 
Fox_TW | Greydwarf | Skeleton | Razorback_TW | 
BlackBear_TW | Greydwarf_Elite | Greydwarf_Shaman | Skeleton_Poison | 
Troll | Ghost | Blob | Draugr | 
Draugr_Ranged | Leech | Surtling | GDAncientShaman_TW | 
Draugr_Elite | Abomination | Ulv | Fenring_Cultist | 
BlobElite | Wraith | Wolf | Hatchling | 
Crawler_TW | HelWraith_TW | StoneGolem | Fenring | 
Serpent | Deathsquito | Lox | Goblin | 
GoblinArcher | RottingElk_TW | GrizzlyBear_TW | GoblinBrute | 
GoblinShaman | ObsidianGolem_TW | Prowler_TW | Seeker | 
SeekerBrute | Dverger | DvergerMage | DvergerMageFire | 
DvergerMageIce | DvergerMageSupport | Gjall | Tick | 
Eikthyr | gd_king | Bonemass | Dragon | 
GoblinKing | SeekerQueen | TreasureChest_meadows | TreasureChest_blackforest | 
TreasureChest_forestcrypt | TreasureChest_fCrypt | TreasureChest_trollcave | shipwreck_karve_chest | 
TreasureChest_meadows_burie

## Generate Mob Loot Tables
### Functions to generate the loot tables for mob tiers and bosses

In [50]:
tiers = set()
for obj in loottable_dict["LootTables"]:
    if "RefObject" in obj.keys():
        tiers.add(obj["RefObject"])
tiers = list(tiers)
tiers.sort()
print(tiers)

['Tier0Mob', 'Tier1Mob', 'Tier2Mob', 'Tier3Mob', 'Tier4Mob', 'Tier5Mob', 'Tier6Mob', 'Tier7Mob']


In [84]:
import numpy as np

def aux_calc(level, tier):
    if tier == 0:
        tier += 1
    return int(np.log2(tier)*1.4) + int(np.log2(level)*1.9) + 1

def prob_calc(total_prob, tier, level, count):
    ret_val = 0
    #print(total_prob)
    if count == 2:
        if total_prob < 0:
            return total_prob
        else:
            ret_val += 1
    try:
        ret_val += pow(tier+1, 0.1) * abs ( total_prob / (level) )
        return abs ( round ( ret_val ) )
    except ZeroDivisionError:
        ret_val += pow(tier+1, 0.1) * abs ( total_prob / (level + 1) )
        return abs( round ( ret_val ) )

def generate_mobs_loot_tables(max_star):
    """
    Generate the loot tables for each mob tier based on the maximum starts fro mobs
    """
    new_loottables = []
    for tier in tiers:
        tier_num = int(tier[4])
        if tier_num < 3:
            rarity = 0
        elif tier_num < 5:
            rarity = 1
        else:
            rarity = int(np.log2(tier_num+1))
        print(f"TIER {tier} - Rarity: {rarity}")
        
        leveled_loot = []
        
        for level in range(1, max_star+1):
            obj = {}
            drops = []
            loot = []
            #########################################################################################
            #                              ASSEMBLING DROPS START                                    
            start_drop_count = max(0, aux_calc(level, tier_num)-2)
            stop_drop_count = aux_calc(level, tier_num)+1
            total_probability = 100
            
            for drop_count in range(start_drop_count, stop_drop_count):
                drops.append([drop_count, 0])
            #########################################################
            while total_probability > 0:
                for i in range(len(drops)):
                    count = drop_count - start_drop_count
                    probability = prob_calc(total_probability, tier_num, level, count)
                    total_probability -= probability
                    drops[i][1] += abs(probability)
            #########################################################
            if total_probability < 0:
                for i in range(len(drops)):
                    if drops[i][1] > 100:
                        drops[i][1] += total_probability
            #########################################################
            if sum([f[1] for f in drops]) > 100:
                drops[0][1] -= (sum([f[1] for f in drops]) - 100)
            #########################################################
            
            #                                ASSEMBLING DROPS END
            #########################################################################################
            obj["Level"] = level
            obj["Drops"] = drops
            obj["Loot"] = [{"Item":f"Tier{rarity}Mats", "Weight":1}]
            leveled_loot.append(obj)
            

            print(obj)
        new_loottables.append({"Object":tier, "LeveledLoot":leveled_loot})
    
    return new_loottables

In [85]:
for tier in tiersgenerate_mobs_loot_tables(8)

TIER Tier0Mob - Rarity: 0
{'Level': 1, 'Drops': [[0, 100], [1, 0]], 'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]}
{'Level': 2, 'Drops': [[0, 56], [1, 30], [2, 14]], 'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]}
{'Level': 3, 'Drops': [[2, 47], [3, 32], [4, 21]], 'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]}
{'Level': 4, 'Drops': [[2, 43], [3, 34], [4, 23]], 'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]}
{'Level': 5, 'Drops': [[3, 40], [4, 33], [5, 27]], 'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]}
{'Level': 6, 'Drops': [[3, 39], [4, 34], [5, 27]], 'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]}
{'Level': 7, 'Drops': [[4, 39], [5, 32], [6, 29]], 'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]}
{'Level': 8, 'Drops': [[4, 37], [5, 34], [6, 29]], 'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]}
TIER Tier1Mob - Rarity: 0
{'Level': 1, 'Drops': [[0, 92], [1, 8]], 'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]}
{'Level': 2, 'Drops': [[0, 59], [1, 28], [2, 13]], 'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}

[{'Object': 'Tier0Mob',
  'LeveledLoot': [{'Level': 1,
    'Drops': [[0, 100], [1, 0]],
    'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]},
   {'Level': 2,
    'Drops': [[0, 56], [1, 30], [2, 14]],
    'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]},
   {'Level': 3,
    'Drops': [[2, 47], [3, 32], [4, 21]],
    'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]},
   {'Level': 4,
    'Drops': [[2, 43], [3, 34], [4, 23]],
    'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]},
   {'Level': 5,
    'Drops': [[3, 40], [4, 33], [5, 27]],
    'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]},
   {'Level': 6,
    'Drops': [[3, 39], [4, 34], [5, 27]],
    'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]},
   {'Level': 7,
    'Drops': [[4, 39], [5, 32], [6, 29]],
    'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]},
   {'Level': 8,
    'Drops': [[4, 37], [5, 34], [6, 29]],
    'Loot': [{'Item': 'Tier0Mats', 'Weight': 1}]}]},
 {'Object': 'Tier1Mob',
  'LeveledLoot': [{'Level': 1,
    'Drops': [[0, 92], [1, 8]],
    'Loot

In [289]:
def prob_calc(total_prob, tier, level, count):
    try:
        return abs ( total_prob // (15 - level + count) )
    except ZeroDivisionError:
        return abs ( total_prob // (14 - level + count) )
    
    
tier_num = int(tiers[1][4])
for level in range(1, 9):
    print("level", level, end=": ")
    remaining_prob = 100
    start_drop_count = max(0, aux_calc(level, tier_num)-2)
    stop_drop_count = aux_calc(level, tier_num)+1
    for drop_count in range(start_drop_count, stop_drop_count):
        #prob = remaining_prob // ( tier_num + level )
        prob = prob_calc(remaining_prob, tier_num, level, drop_count - start_drop_count)
        remaining_prob -= prob
        print(f"{drop_count} ({prob})", end="  ")
    print(f"<{remaining_prob}>")

level 1: 0 (7)  1 (6)  <87>
level 2: 0 (7)  1 (6)  2 (5)  <82>
level 3: 2 (8)  3 (7)  4 (6)  <79>
level 4: 2 (9)  3 (7)  4 (6)  <78>
level 5: 3 (10)  4 (8)  5 (6)  <76>
level 6: 3 (11)  4 (8)  5 (7)  <74>
level 7: 4 (12)  5 (9)  6 (7)  <72>
level 8: 4 (14)  5 (10)  6 (8)  <68>


In [249]:
len("level 7: [6, 10] [7, 10] [8, 10] <70>    ")

41

In [ ]:
mod